# Brain Tumor Detection Data Augmentation

In [4]:

import os
import cv2
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Paths
csv_path = '../brain_tumor/data/mapped_data/mapped_data.csv'
augmented_folder = '../brain_tumor/data/augmented/'
testing_folder = '../brain_tumor/data/testing_data/'

# Create output folders
os.makedirs(augmented_folder, exist_ok=True)
os.makedirs(testing_folder, exist_ok=True)

# Load the CSV
df = pd.read_csv(csv_path)

# Split into 80% train, 20% test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Save test data to separate folder
for _, row in df_test.iterrows():
    image_id = row['Image_ID']
    img_path = row['Image_Path']
    label_path = row['Label_Path']

    if os.path.exists(img_path):
        shutil.copy(img_path, os.path.join(testing_folder, os.path.basename(img_path)))
    if os.path.exists(label_path):
        shutil.copy(label_path, os.path.join(testing_folder, os.path.basename(label_path)))

# Augmentation settings
brightness_factors = [0.6, 0.7, 1.5]
quality_levels = [30, 50]

# Helper to adjust brightness
def adjust_brightness(image, factor):
    return np.clip(image * factor, 0, 255).astype(np.uint8)

# Store new rows
new_rows = []

# Augment only training data
for _, row in df_train.iterrows():
    image_id = row['Image_ID']
    img_path = row['Image_Path']
    label_path = row['Label_Path']

    if not os.path.exists(img_path) or not os.path.exists(label_path):
        continue

    image = cv2.imread(img_path)
    img_ext = os.path.splitext(img_path)[1]

    # Quality augmentation
    for q in quality_levels:
        new_image_id = f"{image_id}_q{q}"
        new_image_name = f"{new_image_id}{img_ext}"
        new_label_name = f"{new_image_id}.txt"
        new_img_path = os.path.join(augmented_folder, new_image_name)
        new_lbl_path = os.path.join(augmented_folder, new_label_name)

        cv2.imwrite(new_img_path, image, [cv2.IMWRITE_JPEG_QUALITY, q])
        shutil.copy(label_path, new_lbl_path)

        new_rows.append({
            'Image_ID': new_image_id,
            'Image_Path': new_img_path,
            'Label_Path': new_lbl_path
        })

    # Brightness augmentation
    for factor in brightness_factors:
        new_image_id = f"{image_id}_b{str(factor).replace('.', '')}"
        new_image_name = f"{new_image_id}{img_ext}"
        new_label_name = f"{new_image_id}.txt"
        new_img_path = os.path.join(augmented_folder, new_image_name)
        new_lbl_path = os.path.join(augmented_folder, new_label_name)

        bright_img = adjust_brightness(image, factor)
        cv2.imwrite(new_img_path, bright_img)
        shutil.copy(label_path, new_lbl_path)

        new_rows.append({
            'Image_ID': new_image_id,
            'Image_Path': new_img_path,
            'Label_Path': new_lbl_path
        })

# Save augmented data
df_augmented = pd.DataFrame(new_rows)
df_final = pd.concat([df_train, df_augmented], ignore_index=True)
df_final.to_csv(csv_path, index=False)

print("Augmentation complete! 80% data augmented, 20% saved to testing_data.")


Augmentation complete! 80% data augmented, 20% saved to testing_data.
